In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
baseurl = 'http://elections.interieur.gouv.fr/presidentielle-2017/'

In [3]:
response = requests.get(baseurl)

In [4]:
doc = BeautifulSoup(response.text, 'html.parser')

In [5]:
links = doc.select('#carte_france area')
len(links)

358

In [6]:
url_list = []
for link in links:
    href = link.get('href')
    if href:
        url_list.append((href, link.get('title')))

In [7]:
url_list = list(set(url_list))

In [8]:
len(url_list)

107

In [19]:
targetCols = ['id', 'title', 'Abstention',
 'Nom',
 'Prenom',
 'Score',
 'Nom_1',
 'Prenom_1',
 'Score_1',
 'Blancs',
 'Statut']

usecols = ['id', 'title', 'Abstention',
 'Candidat1', 'Voix', '% Inscrits', '% Exprimés',
 'Candidat2', 'Voix2', '% Inscrits2', '% Exprimés2', 'Blancs', 'Statut']

In [28]:
df = pd.DataFrame(columns=usecols)

for url in url_list:
    u = url[0]
    print(url[1], '…')
    absoluteURL = baseurl + u.replace('./', '')
    response = requests.get(absoluteURL)
    docDpt = BeautifulSoup(response.text, 'html.parser')
    dpt_nom = re.match('(.*) \(', url[1]).group(1)
    dpt_nom_check = docDpt.select('.pub-fil-ariane a')
    partiel = re.match('.* - (.*)', url[1]).group(1)

    if len(dpt_nom_check) > 2:
        print(dpt_nom, 'vs.', dpt_nom_check[2].string, url)
    
    no_match = re.match('.*\((.*?)\)', url[1])
    dpt_no = no_match.group(1)
    
    result_tables = docDpt.select('.tableau-resultats-listes-ER')

    if len(result_tables) > 1: # si les résultats au 2e tour ont été ajoutés
        rows = result_tables[0].select('tr')
        result_tables = docDpt.select('.tableau-resultats-listes-ER')
        rows = result_tables[0].select('tr')
        print(len(rows), 'rows found in', dpt_nom, ' – ', dpt_no)
        cleanRows = [dpt_no, dpt_nom]

        # 1) abstention
        abstention_tables = docDpt.select('.tableau-mentions')
        td_list = abstention_tables[0].select('td')
        for td in td_list:
            if td.string == 'Abstentions':
                #print(td, td.find_next_siblings()[1].string)
                abstentions = td.find_next_siblings()[1].string.strip()
                cleanRows.append(abstentions)
            elif td.string == 'Blancs':
                print('Blancs found', td.parent())
                blancs = td.find_next_siblings()[1].string.strip()
                
        # 2) candidats
        for row in rows[1:]:
            fields = row.select('td')
            fields_str = [f.string.strip() for f in fields]
            cleanRows.extend(fields_str)

            
        cleanRows.append(blancs)    
        cleanRows.append(partiel)
        df = df.append(pd.DataFrame([(cleanRows)], columns=usecols))


Ariège (09) - résultats complets …
Ariège vs. Ariège (09) ('076/009/index.html', 'Ariège (09) - résultats complets')
3 rows found in Ariège  –  09
Blancs found [<td style="text-align:left">Blancs</td>, <td style="text-align:right">10 147</td>, <td style="text-align:center">   8,64</td>, <td style="text-align:center">  11,19</td>]
Indre (36) - résultats complets …
Indre vs. Indre (36) ('024/036/index.html', 'Indre (36) - résultats complets')
3 rows found in Indre  –  36
Blancs found [<td style="text-align:left">Blancs</td>, <td style="text-align:right">12 583</td>, <td style="text-align:center">   7,38</td>, <td style="text-align:center">   9,64</td>]
Meuse (55) - résultats complets …
Meuse vs. Meuse (55) ('044/055/index.html', 'Meuse (55) - résultats complets')
3 rows found in Meuse  –  55
Blancs found [<td style="text-align:left">Blancs</td>, <td style="text-align:right">9 335</td>, <td style="text-align:center">   6,79</td>, <td style="text-align:center">   8,80</td>]
Rhône (69) - ré

### NB: 79 dpts -> 86 dpts à 23h30 -> 96 à 23h56 -> 103 -> 104 à 0h34 -> 106

In [30]:
df.shape

(106, 13)

In [31]:
def getNom(value):
    if value.find('MACRON') > 0:
        return 'Macron'
    elif value.find('PEN') > 0:
        return 'Le Pen'
    else:
        return False

def getPrenom(value):
    if value.find('MACRON') > 0:
        return 'Emmanuel'
    elif value.find('PEN') > 0:
        return 'Marine'

In [32]:
def cleanNumbers(value):
    return value.replace(' ', '').replace(',', '.')

In [33]:
df_backup = df.copy()

In [34]:
df.head()

,id,title,Abstention,Candidat1,Voix,% Inscrits,% Exprimés,Candidat2,Voix2,% Inscrits2,% Exprimés2,Blancs,Statut
0,09,Ariège,"22,74",M. Emmanuel MACRON,48 015,"40,90","63,09",Mme Marine LE PEN,28 089,"23,92","36,91","8,64",résultats complets
0,36,Indre,"23,43",M. Emmanuel MACRON,68 483,"40,19","60,91",Mme Marine LE PEN,43 954,"25,79","39,09","7,38",résultats complets
0,55,Meuse,"22,88",M. Emmanuel MACRON,48 299,"35,11","51,62",Mme Marine LE PEN,45 271,"32,91","48,38","6,79",résultats complets
0,69,Rhône,"25,08",M. Emmanuel MACRON,572 295,"49,53","73,58",Mme Marine LE PEN,205 467,"17,78","26,42","5,93",résultats complets
0,988,Nouvelle-Calédonie,"47,19",M. Emmanuel MACRON,47 902,"25,30","52,57",Mme Marine LE PEN,43 217,"22,82","47,43","3,45",résultats complets


# 79 -> 86 dpts à 23h30

In [35]:
df['Nom'] = df['Candidat1'].apply(getNom)
df['Prenom'] = df['Candidat1'].apply(getPrenom)

In [36]:
df['Nom_1'] = df['Candidat2'].apply(getNom)
df['Prenom_1'] = df['Candidat2'].apply(getPrenom)

In [38]:
for col in ['Abstention', 'Voix', '% Inscrits',
       '% Exprimés', 'Voix2', '% Inscrits2', '% Exprimés2', 'Blancs']:
    df[col] = df[col].apply(cleanNumbers)

In [39]:
df['Score'] = df['% Exprimés']
df['Score_1']  = df['% Exprimés2']

In [40]:
df['Candidat1'].apply(getNom).value_counts()

Macron    104
Le Pen      2
Name: Candidat1, dtype: int64

In [41]:
def toInt(value):
    if value not in ('2A', '2B'):
        return int(value)
    else:
        return value
df['id'] = df['id'].apply(toInt)

In [42]:
df_noms = pd.read_csv('/Users/paul/Sites/elections_france/d3-composite-projections/data-t1.csv', usecols=['id', 'title'])

In [43]:
df_noms['id'] = df_noms['id'].apply(toInt)

In [44]:
df_noms[df_noms['title'].str.contains('Bourgogne')]

,id,title


In [45]:
dfm = df.merge(df_noms, how='outer', on='title')
del dfm['id_x']
dfm['id'] = dfm['id_y']

In [46]:
dfm

,title,Abstention,Candidat1,Voix,% Inscrits,% Exprimés,Candidat2,Voix2,% Inscrits2,% Exprimés2,Blancs,Statut,Nom,Prenom,Nom_1,Prenom_1,Score,Score_1,id_y,id
0,Ariège,22.74,M. Emmanuel MACRON,48015,40.90,63.09,Mme Marine LE PEN,28089,23.92,36.91,8.64,résultats complets,Macron,Emmanuel,Le Pen,Marine,63.09,36.91,9,9
1,Indre,23.43,M. Emmanuel MACRON,68483,40.19,60.91,Mme Marine LE PEN,43954,25.79,39.09,7.38,résultats complets,Macron,Emmanuel,Le Pen,Marine,60.91,39.09,36,36
2,Meuse,22.88,M. Emmanuel MACRON,48299,35.11,51.62,Mme Marine LE PEN,45271,32.91,48.38,6.79,résultats complets,Macron,Emmanuel,Le Pen,Marine,51.62,48.38,55,55
3,Rhône,25.08,M. Emmanuel MACRON,572295,49.53,73.58,Mme Marine LE PEN,205467,17.78,26.42,5.93,résultats complets,Macron,Emmanuel,Le Pen,Marine,73.58,26.42,69,69
4,Nouvelle-Calédonie,47.19,M. Emmanuel MACRON,47902,25.30,52.57,Mme Marine LE PEN,43217,22.82,47.43,3.45,résultats complets,Macron,Emmanuel,Le Pen,Marine,52.57,47.43,NaN,NaN
5,Ain,22.39,M. Emmanuel MACRON,173832,41.79,60.94,Mme Marine LE PEN,111396,26.78,39.06,6.84,résultats complets,Macron,Emmanuel,Le Pen,Marine,60.94,39.06,1,1
6,Vosges,22.40,M. Emmanuel MACRON,106097,37.55,55.25,Mme Marine LE PEN,85924,30.41,44.75,6.80,résultats complets,Macron,Emmanuel,Le Pen,Marine,55.25,44.75,88,88
7,Polynésie française,53.11,M. Emmanuel MACRON,52378,25.68,58.39,Mme Marine LE PEN,37319,18.30,41.61,1.60,résultats complets,Macron,Emmanuel,Le Pen,Marine,58.39,41.61,NaN,NaN
8,Territoire de Belfort,24.35,M. Emmanuel MACRON,36340,38.14,58.18,Mme Marine LE PEN,26121,27.42,41.82,7.08,résultats complets,Macron,Emmanuel,Le Pen,Marine,58.18,41.82,90,90
9,Saint-Martin/Saint-Barthélemy,64.66,M. Emmanuel MACRON,5282,20.76,65.03,Mme Marine LE PEN,2840,11.16,34.97,2.51,résultats complets,Macron,Emmanuel,Le Pen,Marine,65.03,34.97,NaN,NaN


In [47]:
dfm[targetCols].to_csv('/Users/paul/Sites/elections_france/d3-composite-projections/data-t2.csv', index=False)

In [48]:
print(*dfm[dfm['Abstention'] != dfm['Abstention']]['title'].values, sep=', ')

In [49]:
df.shape

(106, 19)

In [50]:
df['Statut'].value_counts()

résultats complets    106
Name: Statut, dtype: int64